In [ ]:
import hashlib
from io import BytesIO

import util
from test_framework.address import program_to_witness
from test_framework.key import ECKey, ECPubKey, generate_schnorr_nonce
from test_framework.messages import CTransaction, COutPoint, CTxIn, CTxOut, CScriptWitness, CTxInWitness, sha256
from test_framework.musig import generate_musig_key, aggregate_schnorr_nonces, sign_musig, aggregate_musig_signatures
from test_framework.script import TapLeaf, TapTree, CTransaction, TaprootSignatureHash, CScript, ser_string, OP_1, SIGHASH_ALL_TAPROOT
import test_framework.segwit_addr as segwit_addr

# 2.3 Tapscript


In this chapter, we introduce tapscript, an updated Bitcoin scripting language which is introduced in [bip-tapscript](https://github.com/sipa/bips/blob/bip-schnorr/bip-tapscript.mediawiki). Tapscript retains most of the opcodes and evaluation rules applicable to v0 witness scripts, but includes several notable updates described in part 1 of this chapter. The most notable changes in tapscript include signature opcodes which verify bip-schnorr signatures, and the newly added checksigadd signature opcode, which replaces the legacy checkmultisig operation.

* __Part 1: Script updates__
    * Signature opcode updates.
        * Schnorr verification.
        * Checksigadd.
    * Future Versioning
        * Tapscript
        * Opcodes

We also propose a new set of tapscript descriptors in part 2. [Descriptors](https://github.com/bitcoin/bitcoin/blob/cf57e33cc6d95a96f94b259d7680ff9b4f7e22cf/doc/descriptors.md) are a human-readable, high-level template language to describe an individual output or a range of outputs. The proposed Tapscript descriptors include single or multisig (checksigadd) pay-to-pubkey outputs, in combination with hashlocks and time delays.

* __Part 2: Tapscript descriptors__ (Proposed)
    * `Pay-to-pubkey` descriptors
    * `Checksigadd` descriptors

In part 3, we learn how a tapscript can be committed to a taptweak. [Bip-taproot](https://github.com/sipa/bips/blob/bip-schnorr/bip-taproot.mediawiki) proposes such script commitments as an alternative output spending path, which is only revealed when spent. Taproot outputs with committed tapscripts are indistinguishable from other segwit v1 outputs. 

If the internal key of the taproot is a MuSig key, then a committed tapscript is considered an alternative, enforcing spending path, which can impose a separate set of spending conditions independent of the MuSig key. If all participants agree that the locking conditions of the tapscript can be spent, they can collaboratively spend along the MuSig key path, thereby increasing privacy and saving transaction costs.

* __Part 3: Committing scripts into taptweaks__ (Proposed)
    * Tagged Hashing
    * TapLeaf: Tagged tapscript hashes
    * Spending a single tapscript commitment


## Part 1: Script Updates

### Schnorr verification with signature opcodes

The signature opcodes consume the same stack arguments as in Segwit v0, but now verify schnorr signatures as defined in [bip-schnorr](https://github.com/sipa/bips/blob/bip-schnorr/bip-schnorr.mediawiki).

* OP_CHECKSIG
    * Stack arguments consumed: `[public key] [bip-schnorr signature]`
    * Pushes 0x01 on success or 0x00 on failure onto the stack.
    * Requires an empty signature 0x00 to fail.
    
    
* OP_CHECKSIGVERIFY
    * Stack arguments consumed: `[public key] [bip-schnorr signature]`
    * Continues with next opcode or fails script evaluation.

#### 2.3.1 Example: Pay-to-pubkey tapscript.

* The pay-to-pubkey tapscript consist of the following script operations:
    * `[pk] [checksig]`
    
* `TapLeaf.construct_pk(ECPubKey)` contructs a pk tapscript.
* `TapLeaf.script` returns the script opcodes.
* `TapLeaf.sat` returns witness elements required to satisfy the tapscript.

In [ ]:
# Generate key pair
privkey0 = ECKey().generate()
pubkey0 = privkey0.get_pubkey()

# Generate tapscript
pk_tapscript = TapLeaf()
pk_tapscript.construct_pk(pubkey0)

print("Tapscript operations:")
for op in pk_tapscript.script:
    print(op.hex()) if isinstance(op, bytes) else print(op)

print("\nSatisfying witness element:")
for element, value in pk_tapscript.sat:
    print("Witness element type is: {}".format(element))
    print("Signature corresponds to pubkey: {}".format(value.hex()))

### Disabled CHECKMULTISIG opcodes

Legacy k-of-n multisignature opcodes would previously check each of the k signatures against up to n public keys. This is inefficient and disables batch verification of schnorr signatures, since pubkey and signature pairs cannot be known prior to script execution.

* OP_CHECKMULTISIG
* OP_CHECKMULTISIGVERIFY

### Multisignatures with CHECKSIGADD

Tapscript replaces the previous checkmultisig signature operation with OP_CHECKSIGADD. 

This multisignature signature opcode requires the witness to provide a valid or invalid signature for each public key, thereby avoiding the need to waste signature verfication operations for each public key in k-of-n multisignature scripts.

* OP_CHECKSIGADD
    * Equivalent to: `[OP_ROT][OP_SWAP][OP_CHECKSIG][OP_ADD]`
    * Counted as one op towards 201 non-push opcode limit.
    * Enables multisig scripts as shown below.

![test](images/tapscript0.jpg)


### k-of-n CHECKSIGADD tapscripts

Unlike legacy multisig, k-of-n checksigadd multisignature tapscripts will consume a stack element for each public key in the output script. This means that unused public keys must be evaluated against a zero witness element.

For example:
* Tapscript: `[pk0] [CHECKSIG] [PK1] [CHECKSIGADD] [PK2] [CHECKSIGADD] [2] [NUMEQUAL]`
* Possible spending witness(es): 
    * `[sig2]` `[sig1]` `[]`
    * `[sig2]` `[]` `[sig0]`
    * `[]` `[sig1]` `[sig0]`

The disadvantages of k-of-n Checksigadd multisignature scripts include
* Cost: unused public keys are paid by the spender.
* Privacy: unused public keys are revealed when tapscript is spent.

#### 2.3.2 Example: Generating a 2-of-3 checksigadd output

In this example, we construct a 2-of-3 multisig output with `OP_CHECKSIGADD`.

In [ ]:
# Generate key pairs
privkey0 = ECKey().generate()
privkey1 = ECKey().generate()
privkey2 = ECKey().generate()
pubkey0 = privkey0.get_pubkey()
pubkey1 = privkey1.get_pubkey()
pubkey2 = privkey2.get_pubkey()

# Generate tapscript
csa_tapscript = TapLeaf()
csa_tapscript.construct_csa(2, [pubkey0, pubkey1, pubkey2])

print("CSA tapscript operations:")
for op in csa_tapscript.script:
    print(op.hex()) if isinstance(op, bytes) else print(op)

# Satisfying witness element.
print("\nSatisfying witness elements:")
for element, value in csa_tapscript.sat:
    print("Witness element type is: {}".format(element))
    print("Signature corresponds to pubkey: {}".format(value.hex()))

### k-of-k CHECKSIGADD scripts

Alternatively, a k-of-n multisig locking condition can be expressed with multiple k-of-k checksigadd tapscripts. This minimizes leakage of unused public keys and can be more cost-efficient for the spender.

Use the following convenience method to generate k-of-k checksigadd tapscripts from n public keys.
* `TapLeaf.generate_threshold_csa(k, [key_0, key_1, ..., key_n])`

![test](images/tapscript1.jpg)


#### 2.3.3 Example: 2-of-3 multisig expressed as 2-of-2 checksigadd tapscripts.

In [ ]:
# Generate key pairs
privkey0 = ECKey().generate()
privkey1 = ECKey().generate()
privkey2 = ECKey().generate()
pubkey0 = privkey0.get_pubkey()
pubkey1 = privkey1.get_pubkey()
pubkey2 = privkey2.get_pubkey()

# Generate tapscripts
pubkeys = [pubkey0, pubkey1, pubkey2]
tapscripts = TapLeaf.generate_threshold_csa(2, pubkeys)

print("2-of-3 multisig expressed as 2-of-2 checkigadd tapscripts:")
for ts in tapscripts:
    print(ts.desc)

### Tapscript versioning

Tapscript allows for future upgrades of individual tapscripts and specific opcodes.

* Leaf version (commited to TapTree leaf node)
    * Initial version: `0xC0`
    * The leaf version is committed to the tapleaf (See Part 3).
* Success opcodes (allow for future functionality).
    * 80, 98, 126-129, 131-134, 137-138, 141-142, 149-153, 187-254
    * Any of these opcodes end script evaluation successfully.

## Part 2: Tapscript descriptors

A tapscript descriptor is a human-readable language expression which maps to a unique output. We propose each tapscript descriptor to be encapsulated by a tapscript tag `ts`, which can be updated in future tapleaf versions. 
* `ts(pk(key))`, `ts(csa(key))`, ...

### Pay-to-pubkey tapscript descriptors

Next, let us consider specific types of tapscript descriptors. The simplist form of tapscript descriptors are pay-to-pubkey tapscripts. They are spendable by a valid signature witness element, but can be combined with other locking conditions, such as hashlocks and time delays.

* `ts(pk(key))`
    * Witness: `[signature]`
    
    
* `ts(pkhash(key, 20B-hash-digest))`
    * Witness: `[signature]`,`[32B-preimage]`
    
    
* `ts(pkolder(key, delay))`
    * Witness: `[signature]`
    * Spendable after delay (with `nSequence > delay`)
    
    
* `ts(pkhasholder(key, 20B-hash-digest, delay))`
    * Witness: `[signature]`,`[32B-preimage]`
    * Spendable after delay  (with `nSequence > delay`)

We also provide pay-to-pubkey tapscript constructors for for the `TapLeaf` class. 

* `TapLeaf.construct_pk(ECPubKey)`
* `TapLeaf.construct_pkhash(ECPubKey, 20B-hash-digest)`
* `TapLeaf.construct_pkolder(ECPubKey, delay)`
* `TapLeaf.construct_pkhasholder(ECPubKey, 20B-hash-digest, delay)`

The descriptor string can be recalled with:
* `TapLeaf.desc`

**Note:** pubkeys in pay-to-pubkey tapscripts can be generated with multi-party schemes such as MuSig.

#### 2.3.4 Example: Generating a `pkolder` tapscript

We construct a pkhasholder tapscript with the following locking conditions:

* 2-of-2 MuSig public key
* Delay of 20 blocks

In [ ]:
# Generate MuSig key
privkey0 = ECKey().generate()
privkey1 = ECKey().generate()
pubkey0 = privkey0.get_pubkey()
pubkey1 = privkey1.get_pubkey()
c_map, pk_musig = generate_musig_key([pubkey0, pubkey1])

# Generate pkolder tapscript
pkhash_tapscript = TapLeaf()
pkhash_tapscript.construct_pkolder(pk_musig, 20)
print("Tapscript descriptor:", pkhash_tapscript.desc, "\n")

print("Tapscript operations:")
for op in pkhash_tapscript.script:
    print(op.hex()) if isinstance(op, bytes) else print(op)

print("\nSatisfying witness elements:")
for element, value in pkhash_tapscript.sat:
    print("{}, {}".format(element, value.hex()))

### CHECKSIGADD tapscript descriptors

A CHECKSIGADD tapscript descriptor is proposed to have the following forms, and can also be combined with hashlocks and delays.

* `ts(csa(k, [key0, key1, ...]))`
    * Witness: `[signature], [signature], ...`
    * Note: for n < m, empty signature elements (zero) must be provided.
    

* `ts(csaolder(k, [key0, key1, ...], hash))`
    * Witness: `[signature], [signature], ..., [delay]`


* `ts(csahash(k, [key0, key1, ...], hash, time))`
    * Witness: `[signature], [signature], ..., [32B pre-image]`


* `ts(csahasholder(k, [key0, key1, ...], hash, time))`
    * Witness: `[signature], [signature], ..., [32B pre-image], [delay]`

We also provide checksigadd tapscript constructors for for the `TapLeaf` class. 

* `TapLeaf.construct_csa(k, [ECPubKey, ECPubKey, ...])`
* `TapLeaf.construct_csahash(k, [ECPubKey, ECPubKey, ...], 20B-hash-digest)`
* `TapLeaf.construct_csaolder(k, [ECPubKey, ECPubKey, ...], delay)`
* `TapLeaf.construct_csahasholder(k, [ECPubKey, ECPubKey, ...], 20B-hash-digest, delay)`

**Note:** Any single public key in CSA tapscripts can be generated with multi-party schemes such as MuSig.

#### 2.3.5 _Programming Exercise:_ Generate a 2-of-2 `csahasholder` tapscript

Construct a `csahasholder` tapscript with the following locking conditions:

* 2-of-2 public keys
* Hashlock with the preimage `sha256(b'secret')`
* Delay of 20 blocks

In [ ]:
# Generate key pairs
privkey0 = # TODO: implement
privkey1 = # TODO: implement

pubkey0 = # TODO: implement
pubkey1 = # TODO: implement

print("pubkey0: {}".format(pubkey0.get_bytes().hex()))
print("pubkey1: {}\n".format(pubkey1.get_bytes().hex()))

# Method: 32B preimage - sha256('sha256', bytes)
# Method: 20B digest - hashlib.new('ripemd160', bytes).digest()
secret = b'secret'
preimage =  # TODO: implement
digest =  # TODO: implement
delay =  # TODO: implement

# Construct tapscript
csahasholder_tapscript =  # TODO: implement
print("Descriptor:", csahasholder_tapscript.desc, "\n")

print("Tapscript operations:")
for op in csahasholder_tapscript.script:
    print(op.hex()) if isinstance(op, bytes) else print(op)

print("\nSatisfying witness elements:")
for element, value in csahasholder_tapscript.sat:
    print("{}, {}".format(element, value.hex()))

## Part 3: Committing scripts into taptweaks

### Tagged hashes

In chapter 2.2, we learned that it is possible to make valid commitments to a public key, which are called taptweaks. A taptweak commitment is computed with tagged hashes.

* **`TapTweak`** = `sha256(sha256("TapTweak") + sha256("TapTweak") + commitment_hash)`
    
More generally, we can use this function to create tagged hashes, which are described in [bip-schnorr](https://github.com/sipa/bips/blob/bip-schnorr/bip-taproot.mediawiki) and [bip-taproot](https://github.com/sipa/bips/blob/bip-schnorr/bip-taproot.mediawiki). 

* **`tagged_hash("Tag")`** = `sha256(sha256("Tag") + sha256("Tag") + data)`
 
Tagged hashes have the advantages that they provide higher collision resistance. The 64B, duplicated sha256("Tag") expression lends itself to optimization in tagged hash implementations. 

### Single tapleaf script commitments

In order to commit a _tapscript_ to a taptweak, we simply compute the `tagged_hash("TapLeaf")` for the tapscript, along with its tapleaf version and then commit the tapleaf to the taptweak.

* 1. **`TapLeaf`** = `sha256(sha256("TapLeaf") + sha256("TapLeaf") + version|size|script)`
* 2. **`TapTweak`** = `sha256(sha256("TapTweak") + sha256("TapTweak") + internal_pubkey + TapLeaf)`

Initial tapscript version:
* `0xc0`

Script compact size:
* `ser_string(Cscript)` returns the `Cscript` with leading compact size bytes.
* `TapLeaf.script` returns the cscript of the tapscript/tapleaf.

#### _2.3.6 Programming Exercise:_ Compute the taptweak from a tapscript

* Use the `tagged_hash` method to compute a tagged hash.
* Generate an internal public key.
* Compute the taptweak from a single `csahasholder_tapscript` commitment.

In [ ]:
def tagged_hash(tag, input_data):
    data = sha256(tag.encode('utf-8'))
    data += data
    data += input_data
    return sha256(data)

privkey_internal =  # TODO: implement
pubkey_internal =  # TODO: implement

# Method: ser_string(Cscript) prepends compact size.
TAPSCRIPT_VER = bytes([0xc0])
tapleaf =  # TODO: implement
taptweak =  # TODO: implement
print("Your constructed taptweak is: {}.".format(taptweak.hex()))

#### 2.3.7 Example: Compare tagged hash with taptweak constructor

* The `TapTree.construct()` method constructs the triple: `segwit_v1_cscript`, `taptweak`, `cblock_map`.
* Run the code below to generate the taptweak and compare with your taptweak computation.

In [ ]:
taptree = TapTree()
taptree.key = pubkey_internal
taptree.root = csahasholder_tapscript
segwit_v1_script, tap_tweak_constructed, control_map = taptree.construct()

assert taptweak == tap_tweak_constructed
print("Success! Your constructed taptweak is correct.")

### Spending a single tapscript script commitment

The witness which can spend a single committed tapscript requires witness data which provides the satifying elements of the tapscript, and proof that the tapscript is committed to the witness program.

* `[Stack element(s) satisfying tapscript]`
* `[Tapscript]`
* `[Controlblock c]`

The controlblock c is a single stack element consisting of:
* `[Tapscript version]`
    * `0xfe & c[0]`
* `[Internal Public Key]`
    * `y-coordinate: 0x01 & c[0]`
    * `x-coordinate: c[1:33]`



#### Example 2.3.8:  Generate a single tapscript segwit v1 address

In this example, we construct segwit v1 output for spending along the single script path. We will reuse the previosuly generated segwit v1 witness program which has the `csahasholder` tapscript committed to it, and encode it to a bech32 address.

In [ ]:
# Tweak the internal key to obtain the Segwit program 
# ([1B oddness] [32B x-coordinate])
taproot_pubkey = pubkey_internal.tweak_add(taptweak) 
taproot_pubkey_b = taproot_pubkey.get_bytes()
program = bytes([taproot_pubkey_b[0] & 1]) + taproot_pubkey_b[1:]
print("Witness program is {}\n".format(program.hex()))

# Create (regtest) bech32 address
version = 0x01
address = segwit_addr.encode("bcrt", version, program)
print("bech32 address is {}".format(address))

#### Example 2.3.9: Startup TestWrapper to initialize a regtest node and wallet

Run setup only once, or after a clean shutdown.

In [ ]:
test = util.TestWrapper()
test.setup(num_nodes=1)

version = test.nodes[0].getnetworkinfo()['subversion']
print("Client version is {}".format(version))

####  Example 2.3.10: Generate coins for the wallet

In [ ]:
test.nodes[0].generate(101)
balance = test.nodes[0].getbalance()
print('Balance: {}'.format(balance))

assert balance > 1

#### Example 2.3.11: Send funds from the Bitcoin Core wallet to the segwit v1 address

In [ ]:
# Send wallet transaction to segwit address
amount_btc = 0.05
txid = test.nodes[0].sendtoaddress(address, amount_btc)

# Decode wallet transaction
tx_hex = test.nodes[0].getrawtransaction(txid) 
decoded_tx = test.nodes[0].decoderawtransaction(tx_hex)

print("Transaction:\n{}\n".format(decoded_tx))

# Reconstruct wallet transaction locally
tx = CTransaction()
tx.deserialize(BytesIO(bytes.fromhex(tx_hex)))
tx.rehash()

# We can check if the transaction was correctly deserialized
assert txid == decoded_tx["txid"]

# The wallet randomizes the change output index for privacy
# Loop through the outputs and return the first where the scriptPubKey matches the segwit v1 output
output_index, output = next(out for out in enumerate(tx.vout) if out[1].scriptPubKey == CScript([OP_1, program]))

print("Segwit v1 output is {}".format(output))
print("Segwit v1 output value is {}".format(output.nValue))
print("Segwit v1 output index is {}".format(output_index))

#### _Programming Exercise 2.3.12:_ Construct `CTransaction` and populate inputs

* Transaction version must set to 2 if the tapscript has set a spend delay.
* The input sequence must be encoded with the required spend delay.
    * `CTxIn(outpoint = ..., nSequence = ...)`

In [ ]:
# Construct transaction
spending_tx = CTransaction()

# Populate the transaction version
spending_tx.nVersion =  # TODO: implement

# Populate the locktime
spending_tx.nLockTime = 0

# Populate the transaction inputs
# Method: Construct COutPoint(txid, index)
# Method: Construct CTxIn(outpoint = ..., nSequence = ...)
# Tip: CTransaction.vin = "list of CTxIn objects"
spending_tx.vin =  # TODO: implement

print("Spending transaction:\n{}".format(spending_tx))

#### Example 2.3.13: Populate outputs

We'll generate an output address in the Bitcoin Core wallet to send the funds to, determine the fee, and then populate the spending tx with an output to that address.

In [ ]:
# Generate new Bitcoin Core wallet address
dest_addr = test.nodes[0].getnewaddress(address_type="bech32")
scriptpubkey = bytes.fromhex(test.nodes[0].getaddressinfo(dest_addr)['scriptPubKey'])

# Determine minimum fee required for mempool acceptance
min_fee = int(test.nodes[0].getmempoolinfo()['mempoolminfee'] * 100000000)

# Complete output which returns funds to Bitcoin Core wallet
amount_sat = int(amount_btc * 100000000)
dest_output = CTxOut(nValue=amount_sat - min_fee, scriptPubKey=scriptpubkey)
spending_tx.vout = [dest_output]

print("Spending transaction:\n{}".format(spending_tx))

#### _Programming Exercise 2.3.14:_ Sign the transaction

Note that we must pass the following arguments to `TaprootSignatureHash` for script path spending:
* `scriptpath`: `True`
* `tapscript`: `Cscript` of tapscript

In [ ]:
# Generate the Taproot Signature Hash for signing
sighash = TaprootSignatureHash(spending_tx,
                               [output],
                               SIGHASH_ALL_TAPROOT,
                               input_index = 0,
                               scriptpath =  # TODO: implement
                               tapscript =  # TODO: implement

# Sign with both privkeys
signature0 =  # TODO: implement
signature1 =  # TODO: implement

print("Signature0: {}".format(signature0.hex()))
print("Signature1: {}".format(signature1.hex()))

#### _Programming Exercise 2.3.15:_ Add the witness and test acceptance of the transaction

Remember to revisit the satisfying witness elements for `csahasholder_tapscript` constructed in exercise 2.3.5:
* Preimage
* Signature for pubkey1
* Signature for pubkey0

Ensure that the time-lock performs as expected.

In [ ]:
# Construct transaction witness
# Tip: Witness stack for script path - [satisfying elements for tapscript] [TapLeaf.script] [controlblock]
# Tip: Controlblock for a tapscript in control_map[TapLeaf.script]
witness = CScriptWitness()
witness.stack =  # TODO: implement
witness_in = CTxInWitness()
witness_in.scriptWitness = witness
spending_tx.wit.vtxinwit.append(witness_in)

print("Spending transaction:\n{}\n".format(spending_tx))

# Serialize signed transaction for broadcast
spending_tx_str = spending_tx.serialize().hex()
 
# Test mempool acceptance with and without delay.
assert test.nodes[0].testmempoolaccept([spending_tx_str])[0]['allowed']
assert not test.nodes[0].testmempoolaccept([spending_tx_str])[0]['allowed']
test.nodes[0].generate(delay)
assert test.nodes[0].testmempoolaccept([spending_tx_str])[0]['allowed']

print("Success!")

#### Shutdown TestWrapper

In [ ]:
# Shutdown
test.shutdown()

**Congratulations!** In this chapter, you have:

- Learned how the tapscript semantics differ from legacy Bitcoin script semantics:
    - `OP_CHECKSIG` opcodes verify bip-schnorr signatures instead of ecdsa signatures.
    - `OP_CHECKMULTISIG` and `OP_CHECKMULTISIGVERIFY` are replaced by `OP_CHECKSIGADD`.
- Converted a k-of-n threshold signing scheme into a tree of multiple k-of-k threshold signing schemes.
- Learned how tapscript is versioned.
- Used output descriptors to specify and construct a tapscript.
- Learned how a tapscript can be committed as a tweak into a taproot internal public key.
- Sent coins to a segwit v1 output with a tweaked public key committing to a tapscript, and later spent that output by using the script path.